In [8]:

import kaggle
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
orders.csv: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
#Import Data From zip file to csv file
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall()
zip_ref.close()

In [10]:
# Read Data From File And Handle Null Values

import pandas as pd
df = pd.read_csv('orders.csv', na_values = ['Not Available','unknown'])
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [11]:
# Rename Columns Name And Make Then Into Lower Case And Replace With Underscore
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(" ","_")


In [12]:
# Derive New Column discount on Sale Price And Discount Percent
df['discount'] = df['list_price']*df['discount_percent']*.01 
df['sale_price'] = df['list_price']-df['discount']
df['profit'] = df['sale_price']-df['cost_price']


In [13]:
#Change Order_date To Datetime Type
df['order_date']= pd.to_datetime(df['order_date'],format='%Y-%m-%d')


In [14]:
# Drop unnecessary columns
df = df.drop(columns = ['list_price','cost_price','discount_percent'])

In [15]:
import sqlalchemy as sal
connection_url = (
    "mssql+pyodbc://@MSI\\SQLEXPRESS/order_analysis"
    "?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)

engine = sal.create_engine(connection_url)
conn = engine.connect()
print("Connected successfully!")

Connected successfully!


In [18]:
# Get existing order_id values from the SQL table
existing_ids = pd.read_sql("SELECT order_id FROM df_orders", conn)

# Filter out rows from df that are already in the table
df_filtered = df[~df['order_id'].isin(existing_ids['order_id'])]

# Insert only the new data
df_filtered.to_sql('df_orders', con=conn, index=False, if_exists='append')


0

In [19]:
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'quantity', 'discount', 'sale_price', 'profit'],
      dtype='object')